# Importing Libraries

In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

ModuleNotFoundError: No module named 'requests'

# Load environment & Check the key

In [ ]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
openai = OpenAI()

if api_key:
    print("API key found and looks good so far!")
else:
    print("API key not found. Please check your .env file.")

API key found and looks good so far!


# Define Class to represent Webpage

In [ ]:
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        self.title = soup.title.string if soup.title else "there is no title"
        for irrelevent in soup.body(["script", "style", "img", "input"]):
            irrelevent.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
ed = Website(
    "https://learn.microsoft.com/en-us/dynamics365/commerce/channel-setup-retail"
)
print(ed.title)
# print(ed.text)

Set up a retail  channel - Commerce | Dynamics 365 | Microsoft Learn


In [ ]:
system_prompt = """
You are an assistant tasked with analyzing a website's content and offering a brief summary. 
Avoid including text that seems related to navigation. Reply in markdown format.
"""

In [ ]:
def user_prompt_for(Website):
    user_prompt = f"the Website title is {Website.title}"
    user_prompt += """
    The contents of this website is as follows; \
    please provide a short summary of this website in markdown. \
    If it includes news or announcements, then summarize these too.\n\n
    """
    user_prompt += Website.text
    return user_prompt

The API from OpenAI expects to receive messages in a particular structure.  
Many of the other APIs share this structure:  

[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]


In [ ]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)},
    ]

In [ ]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model="gpt-4o-mini", messages=messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
summarize("https://learn.microsoft.com/en-us/dynamics365/commerce/channel-setup-retail")

'# Summary of "Set up a retail channel - Commerce | Dynamics 365 | Microsoft Learn"\n\nThis article provides a comprehensive guide on creating a new retail channel using Microsoft Dynamics 365 Commerce. It highlights the different types of retail channels available, such as online stores, call centers, and physical retail outlets. \n\n## Key Points:\n\n- **Channel Setup Requirements**: Before establishing a retail channel, certain prerequisites must be met, including configuring payment methods, setting up income and expense accounts, and customizing staff roles.\n\n- **Step-by-Step Configuration**: The guide details the process for configuring a new retail channel, including fields such as channel name, store number, legal entity, currency, and more. \n\n- **Additional Setup Items**: Instructions for setting up crucial elements like payment methods, cash declarations, delivery modes, income and expense accounts, and fulfillment group assignments are provided. \n\n- **Transaction ID Ma

In [ ]:
# A function to display this nicely in the Jupyter output, using markdown


def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary(
    "https://learn.microsoft.com/en-us/dynamics365/commerce/channel-setup-retail"
)

# Summary of "Set up a retail channel - Commerce | Dynamics 365 | Microsoft Learn"

The webpage provides a detailed guide on how to create and configure new retail channels within Microsoft Dynamics 365 Commerce. It outlines the various types of retail channels, including online stores, call centers, and brick-and-mortar stores. Each channel can have distinct settings such as payment methods, pricing structures, and point-of-sale configurations.

## Key Instructions

- **Creating a Retail Channel**: Users are guided to navigate to the specific modules in Dynamics 365, input key information (like store name, number, time zone, and tax group), and save the configurations.
  
- **Additional Settings**: The guide discusses setting up payment methods, cash declarations, modes of delivery, income/expense accounts, sections, and fulfillment group assignments.

- **Transaction IDs**: A new format for transaction IDs is introduced to prevent duplication, combining store ID and terminal ID with a timestamp. This new method becomes effective in Commerce version 10.0.41.

- **Store Location**: Instructions are given for inputting geographical data, which supports store selection functionality in e-commerce platforms and POS systems.

## Additional Information
The article is regularly updated, indicating a specific last modification date of October 29, 2024, and encourages user feedback regarding page helpfulness.

This resource is crucial for developers and administrators looking to effectively manage and set up retail operations within Dynamics 365 Commerce.